In [ ]:

for game_num in range(len(sample_games))[0:1]:
    pbp = pd.read_csv('Basketball Analytics/Play_by_Play.txt',delimiter='\t')
    lineup = pd.read_csv('Basketball Analytics/Game_Lineup.txt',delimiter='\t')
    codes = pd.read_csv('Basketball Analytics/Event_Codes.txt',delimiter = '\t')

    game = sample_games[game_num] # for now, not iterative.
    print("game id: ", game)
    print("game num: ",game_num)
    teams = lineup.loc[lineup['Game_id'] == game]['Team_id'].unique() #locate the two teams in the game. 
    
    
    #sort according to this, it may end up changing . 
    pbp_singlegame = pbp.loc[pbp['Game_id'] == sample_games[game_num]] #.sort_values(['Period','PC_Time','WC_Time','Event_Num'],
       # ascending=[True,False,True,True])
    
    #translate using codes
    pbp_singlegame = pbp_singlegame.merge( codes,
        on = ['Event_Msg_Type', 'Action_Type'], how = 'left')
    

    #retain every event that is a made shot
    pbp_singlegame = pbp_singlegame.loc[(pbp_singlegame['Event_Msg_Type'] == 1) |
        (pbp_singlegame['Event_Msg_Type'] == 3) | (pbp_singlegame['Event_Msg_Type'] == 8) |
        (pbp_singlegame['Event_Msg_Type'] == 12) | (pbp_singlegame['Event_Msg_Type'] == 13)]
    

    #delete this extraneous col
   # del pbp_singlegame['Action_Type_Description']
    

    #obtain starting lineups
    starting_lineup = lineup.loc[(lineup['Game_id'] == game)] #starting lineup of the game
    
    subsmask = pbp_singlegame['Event_Msg_Type'] == 8
    subs = pbp_singlegame[subsmask] #dataframe of all substitutions. 
    
    
    playersin = pd.DataFrame(starting_lineup.loc[(starting_lineup['Period'] == 1),['Team_id','Person_id']])
    subs_correct = substitution_correction(subs,starting_lineup)
    pbp_singlegame[subsmask] =  subs_correct #fixes substitutions, and cofnirms taem names are correct. 
    subs = pbp_singlegame[subsmask] 
    pbp_singlegame = pbp_singlegame.sort_values(['Period','PC_Time','WC_Time','Event_Num'],
        ascending=[True,False,True,True])
    
    pbp_singlegame = possession_flagger(pbp_singlegame)
    pbp_singlegame = score_aggregator(pbp_singlegame)
    
    #Initialize point differential both game and players for the dataset. 
    playersin['diffin'] = playersin['pm'] = playersin['plusin'] = playersin['opts'] = 0
    playersin['minusin'] = playersin['dpts'] =0
    bench = pd.DataFrame(columns = ['Team_id', 'Person_id', 'diffin', 'pm','plusin','opts','minusin','dpts'])
    #
    for index, row in pbp_singlegame.iterrows():
       # print()
       # print(index,row)
        if (row['Event_Msg_Type'] == 8):
            playersin, bench = sub(playersin, bench, row)  #calculate +/- of subout.
        #    playersin, suboutindex,score = sub(playersin, bench, row)  #calculate +/- of subout.
        elif (row['Event_Msg_Type'] == 13):
           # i+=1
    
            playersin, bench = endperiod(playersin, bench, row)  #calculate +/- at end of period,


        elif (row['Event_Msg_Type'] == 12):
            playersin, bench = startperiod(playersin, bench, row) #update lineups



In [1]:
from nba_py import player

In [5]:
from nba_py import league

In [21]:
from nba_py import game

In [14]:
league.PlayerStats().overall().columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'GP',
       'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV',
       'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'GP_RANK', 'W_RANK', 'L_RANK',
       'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK',
       'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK',
       'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK',
       'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK',
       'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK',
       'TD3_RANK', 'CFID', 'CFPARAMS'],
      dtype='object')

In [20]:
player.PlayerList(season='2007-08').info()

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
0,949,"Abdur-Rahim, Shareef",Shareef Abdur-Rahim,1,1996,2007,shareef_abdur-rahim,1610612758,Sacramento,Kings,SAC,kings,Y,00
1,200984,"Allred, Lance",Lance Allred,1,2007,2007,lance_allred,1610612739,Cleveland,Cavaliers,CLE,cavaliers,Y,11
2,1507,"Anderson, Derek",Derek Anderson,1,1997,2007,derek_anderson,1610612766,Charlotte,Bobcats,CHA,hornets,Y,00
3,353,"Armstrong, Darrell",Darrell Armstrong,1,1994,2007,darrell_armstrong,1610612751,New Jersey,Nets,NJN,nets,Y,00
4,200788,"Augustine, James",James Augustine,1,2006,2007,james_augustine,1610612753,Orlando,Magic,ORL,magic,Y,01
5,2857,"Barrett, Andre",Andre Barrett,0,2004,2007,andre_barrett,0,,,,,Y,11
6,2810,"Brown, Andre",Andre Brown,1,2006,2008,andre_brown,1610612763,Memphis,Grizzlies,MEM,grizzlies,Y,01
7,136,"Brown, P.J.",P.J. Brown,1,1993,2007,p.j._brown,1610612738,Boston,Celtics,BOS,celtics,Y,00
8,208,"Cassell, Sam",Sam Cassell,1,1993,2007,sam_cassell,1610612738,Boston,Celtics,BOS,celtics,Y,00
9,1506,"Croshere, Austin",Austin Croshere,1,1997,2008,austin_croshere,1610612744,Golden State,Warriors,GSW,warriors,Y,00


In [35]:
game.BoxscoreAdvanced(game_id='0041700106',season='2017-18',season_type='Playoffs').sql_players_advanced().columns



Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'E_OFF_RATING',
       'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING',
       'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT', 'DREB_PCT',
       'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT',
       'E_PACE', 'PACE', 'PIE'],
      dtype='object')

In [36]:
game.BoxscoreAdvanced(game_id='0041700106',season='2017-18',season_type='Playoffs').sql_team_advanced().columns


Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PIE'],
      dtype='object')

In [39]:
game.Boxscore(game_id='0041700106',season='2017-18',season_type='Playoffs').player_stats()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0041700106,1610612761,TOR,Toronto,1628384,OG Anunoby,F,,20:05,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,3.0
1,0041700106,1610612761,TOR,Toronto,201586,Serge Ibaka,F,,22:37,3.0,...,1.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,7.0,-5.0
2,0041700106,1610612761,TOR,Toronto,202685,Jonas Valanciunas,C,,30:55,6.0,...,6.0,6.0,12.0,0.0,1.0,2.0,1.0,3.0,14.0,6.0
3,0041700106,1610612761,TOR,Toronto,201942,DeMar DeRozan,G,,33:27,6.0,...,0.0,2.0,2.0,4.0,0.0,0.0,0.0,2.0,16.0,-6.0
4,0041700106,1610612761,TOR,Toronto,200768,Kyle Lowry,G,,31:37,9.0,...,1.0,5.0,6.0,6.0,1.0,0.0,2.0,2.0,24.0,0.0
5,0041700106,1610612761,TOR,Toronto,1627751,Jakob Poeltl,,,17:05,2.0,...,3.0,4.0,7.0,1.0,0.0,0.0,0.0,5.0,7.0,4.0
6,0041700106,1610612761,TOR,Toronto,101139,CJ Miles,,,19:03,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,7.0,6.0
7,0041700106,1610612761,TOR,Toronto,1627783,Pascal Siakam,,,22:08,5.0,...,3.0,5.0,8.0,0.0,1.0,2.0,0.0,3.0,11.0,18.0
8,0041700106,1610612761,TOR,Toronto,1626153,Delon Wright,,,24:17,2.0,...,2.0,2.0,4.0,2.0,4.0,1.0,1.0,0.0,4.0,12.0
9,0041700106,1610612761,TOR,Toronto,1627832,Fred VanVleet,,,18:46,2.0,...,0.0,4.0,4.0,4.0,0.0,0.0,1.0,1.0,5.0,12.0


In [41]:
game.Boxscore(game_id='0041700106',season='2017-18',season_type='Playoffs').team_stats().columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS'],
      dtype='object')

In [45]:
game.PlayByPlay(game_id='0041700106').available_video()


In [46]:
game.PlayByPlay(game_id='0041700106').info()


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0041700106,2,12,0,1,7:11 PM,12:00,None,None,None,None,None
1,0041700106,4,10,0,1,7:11 PM,12:00,Jump Ball Gortat vs. Valanciunas: Tip to Wall,None,None,None,None
2,0041700106,7,1,79,1,7:11 PM,11:42,Mark Morris 14' Pullup Jump Shot (2 PTS),None,None,0 - 2,2
3,0041700106,8,6,2,1,7:11 PM,11:42,None,None,Ibaka S.FOUL (P1.T1) (S.Corbin),None,None
4,0041700106,10,3,10,1,7:12 PM,11:42,Mark Morris Free Throw 1 of 1 (3 PTS),None,None,0 - 3,3
5,0041700106,11,1,79,1,7:12 PM,11:30,None,None,DeRozan 19' Pullup Jump Shot (2 PTS),2 - 3,1
6,0041700106,12,2,63,1,7:12 PM,11:08,MISS Mark Morris 12' Fadeaway Jumper,None,None,None,None
7,0041700106,13,4,0,1,7:12 PM,11:06,None,None,Ibaka REBOUND (Off:0 Def:1),None,None
8,0041700106,14,2,41,1,7:12 PM,11:01,Gortat BLOCK (1 BLK),None,MISS Lowry 3' Running Layup,None,None
9,0041700106,15,4,0,1,7:12 PM,11:01,Mark Morris REBOUND (Off:0 Def:1),None,None,None,None


In [26]:
Scoreboard().game_header()

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS


In [29]:
Scoreboard(month=4, day=27, year=2018).game_header()

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS
0,2018-04-27T00:00:00,2,0041700106,3,Final,20180427/TORWAS,1610612764,1610612761,2017,4,,ESPNews,Q4 - ESPNews,1
1,2018-04-27T00:00:00,3,0041700136,3,Final,20180427/CLEIND,1610612754,1610612739,2017,4,,ESPN,Q4 - ESPN,1
2,2018-04-27T00:00:00,4,0041700176,3,Final,20180427/OKCUTA,1610612762,1610612760,2017,4,,ESPN,Q4 - ESPN,1
